In [1]:
import pandas as pd
from read_data import read_data

def compute_delta_metrics(data):
    """
    Computes additional metrics for the dataset:
    - Delta Position: Leader's position minus Follower's position.
    - Delta Velocity: Leader's velocity minus Follower's velocity.
    - Delta Acceleration: Leader's acceleration minus Follower's acceleration.
    - Time-To-Collision (TTC): Delta Position divided by Delta Velocity.
    """
    data["delta_position"] = data["x_leader"] - data["x_follower"]
    data["delta_velocity"] = data["v_leader"] - data["v_follower"]
    data["delta_acceleration"] = data["a_leader"] - data["a_follower"]
    data["TTC"] = data["delta_position"] / data["delta_velocity"]
    return data

def aggregate_data_by_case(data):
    """
    Aggregates the dataset by 'case_id' to find the max and min 
    of each delta metric and TTC for each case.
    Renames columns for clarity and adds case_id as a column.
    """
    aggr_data = data.groupby('case_id').agg({
        'delta_position': ['max', 'min'], 
        'delta_velocity': ['max', 'min'], 
        'delta_acceleration': ['max', 'min'], 
        'TTC': ['max', 'min']
    })
    aggr_data.columns = ["_".join(x) for x in aggr_data.columns.ravel()]
    aggr_data["case_id"] = aggr_data.index
    aggr_data.columns = [
        'max_delta_position', 'min_delta_position', 
        'max_delta_velocity', 'min_delta_velocity', 
        'max_delta_acceleration', 'min_delta_acceleration', 
        'max_TTC', 'min_TTC', 'case_id'
    ]
    aggr_data = aggr_data[
        ['case_id', 'max_delta_position', 'min_delta_position', 
        'max_delta_velocity', 'min_delta_velocity', 
        'max_delta_acceleration', 'min_delta_acceleration', 
        'max_TTC', 'min_TTC']
    ]
    return aggr_data

def adjust_ttc_sign(aggregated_data):
    """
    Ensures TTC (Time-To-Collision) is non-negative by taking the absolute value.
    """
    aggregated_data["min_TTC"] = aggregated_data["min_TTC"].abs()
    return aggregated_data

def convert_df(dataset: str, mode: str):
    """
    Main function that utilizes the above helper functions to preprocess the data.
    Returns a DataFrame grouped by 'case_id' with max and min values of
    delta position, delta velocity, delta acceleration, and TTC (Time-To-Collision).
    """
    data = read_data(dataset, mode)
    data = compute_delta_metrics(data)
    aggregated_data = aggregate_data_by_case(data)
    aggregated_data = adjust_ttc_sign(aggregated_data)
    return aggregated_data


In [2]:
trainHA = convert_df("HA", "train")
trainHA.describe()

,case_id,max_delta_position,min_delta_position,max_delta_velocity,min_delta_velocity,max_delta_acceleration,min_delta_acceleration,max_TTC,min_TTC
count,26394.000000,26394.000000,26394.000000,26394.000000,26394.000000,26394.000000,26394.000000,2.639400e+04,2.639400e+04
mean,13196.500000,25.333803,12.660995,1.907915,-1.961766,1.214429,-1.377007,1.946642e+05,2.023333e+05
std,7619.435839,11.389465,7.005050,1.246547,1.262682,0.625144,1.297498,1.134597e+07,1.763010e+07
min,0.000000,7.970585,3.192059,-2.645687,-15.422088,-0.164731,-10.560531,-5.370677e+01,2.865257e+00
25%,6598.250000,18.117176,7.330479,0.874794,-2.585847,0.716436,-1.457096,1.228487e+03,1.226827e+03
50%,13196.500000,22.091704,11.508556,1.803440,-1.798556,1.164713,-1.044787,3.797345e+03,3.719639e+03
75%,19794.750000,28.647320,15.768843,2.779035,-1.054550,1.601606,-0.748433,1.528928e+04,1.520460e+04
max,26393.000000,85.211403,78.518199,13.742090,1.298914,6.417317,-0.032989,1.579993e+09,2.840646e+09


In [3]:
df = trainHA.head(5000)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import math

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
    
    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(data):
            break
        X.append(data[i:end_ix, :])
        y.append(data[end_ix:out_end_ix, :])
    return np.array(X), np.array(y)

def preprocess_data(df, n_steps_in, n_steps_out, test_size=0.2):
    scaler = MinMaxScaler()
    data_normalized = scaler.fit_transform(df.values)
    X, y = create_sequences(data_normalized, n_steps_in, n_steps_out)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return (
        torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32),
        torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32),
        scaler
    )

def train_model(model, X_train_tensor, y_train_tensor, epochs, optimizer, loss_function):
    for epoch in tqdm(range(epochs)):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = loss_function(y_pred, y_train_tensor[:,0,:])
        loss.backward()
        optimizer.step()
        if epoch%10 == 0:
            tqdm.write(f'epoch: {epoch} loss: {loss.item():.8f}')

def evaluate_model(model, X_test_tensor, y_test_tensor, scaler):
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_test_tensor)
        y_test_pred_np = scaler.inverse_transform(y_test_pred.numpy())
        y_test_actual_np = scaler.inverse_transform(y_test_tensor[:, 0, :].numpy())
        
        mse = mean_squared_error(y_test_actual_np, y_test_pred_np)
        rmse = math.sqrt(mse)
        mae = mean_absolute_error(y_test_actual_np, y_test_pred_np)
    
    print(f'Mean Squared Error (MSE): {mse:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    
    return mse, rmse, mae

# Usage
# Assume df is your DataFrame
n_steps_in, n_steps_out = 3, 1
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, scaler = preprocess_data(df, n_steps_in, n_steps_out)

model = LSTMModel(input_size=X_train_tensor.shape[2], hidden_layer_size=50, output_size=X_train_tensor.shape[2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()

train_model(model, X_train_tensor, y_train_tensor, epochs=100, optimizer=optimizer, loss_function=loss_function)
evaluate_model(model, X_test_tensor, y_test_tensor, scaler)


  5%|▌         | 5/100 [00:00<00:04, 22.62it/s]

epoch: 0 loss: 0.25866872


 17%|█▋        | 17/100 [00:00<00:02, 32.23it/s]

epoch: 10 loss: 0.19594710


 25%|██▌       | 25/100 [00:00<00:02, 30.76it/s]

...

 95%|█████████▌| 95/100 [00:03<00:00, 31.46it/s]

epoch: 90 loss: 0.01554240


100%|██████████| 100/100 [00:03<00:00, 30.87it/s]

Mean Squared Error (MSE): 148191870976.00
Root Mean Squared Error (RMSE): 384956.97
Mean Absolute Error (MAE): 41951.61


(148191870000.0, 384956.9728891789, 41951.61)

In [5]:
eval_df = convert_df("HA", "val")


In [7]:
model

LSTMModel(
  (lstm): LSTM(9, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=9, bias=True)
)